The following cell sets the input values required to run the the optimisation model ``..\MINIZINC\hydrogen_plant.mzn``.

This input data are stored in ``..\MINIZINC\hydrogen_plant_data.dzn``. This input file preparation is carried out by ``make_dzn_file`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``.

Then the minizinc optimisation is executed by ``optimise()`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``. 

``projdirs`` stores the key paths for the entire package.

In [1]:
import pandas as pd
import numpy as np
import os
from projdirs import datadir #load the path that contains the data files 
from PACKAGE.optimisation import make_dzn_file, optimise
from PACKAGE.component_model import WindSource, SolarResource,pv_gen, wind_gen

# Set the location
Lat = -22.959379
Lon = 117.812731


# Get wind and solar data for the designated location
WindSource(Lat, Lon)
SolarResource(Lat,Lon)


#Load the weather data file and extract wind speed and solar irradiance from it
DATA = pd.read_csv(datadir + r'TEST DATA\SAM_60min\086282TMY_60min_SAM.csv',skiprows=2)
# solar = DATA['global horizontal irradiance'].copy().tolist()
PV_ref_capa = 1000000 #(W)
PV_ref_out = pv_gen(PV_ref_capa)
wind = DATA['wind speed'].copy().tolist()

# create a dictionary that contains the inputs for optimisation.
#these inputs are used by make_dzn_file function to create an input text file called hydrogen_plant_data.dzn. 
simparams = dict(DT = 3600,#[s] time steps
                 ETA_EL = 0.80,       #efficiency of electrolyser
                 BAT_ETA_in = 0.95,   #charging efficiency of battery
                 BAT_ETA_out = 0.95,  #discharg efficiency of battery
                 C_PV = 0.95,          #[$/W] unit cost of PV
                 C_W = 0.7,           #[$/W] unit cost of W
                 C_EL = 0.5,          #[$/W] unit cost of electrolyser
                 C_HS = 16,           #[$/kgH] unit cost of hydrogen storage
                 C_BAT = 200/1000/3600,        #[$/W.s] unit cost of battery storage
                 CF = 0.99,           #capacity factor
                 pv_ref_capa = PV_ref_capa,    #capacity of reference PV plant (W)
                 pv_ref_out = PV_ref_out,           #power output from reference PV plant (W)
                 W = [1*w for w in wind],            #wind speed timeseries
                 L = [0.1 for i in range(len(wind))])       #[kgH/s] load profile timeseries

#run the optimisation function and get the results in a dictionary:
results = optimise(simparams)


print (
        'CAPEX = %0.2f [M$]' %( results['CAPEX'][0]/1e6 ),'\n'
        'pv_max = %0.2f [MW]' %( results['pv_max'][0]/1e6 ),'\n'  
        'w_max = %0.2f [MW]' %( results['w_max'][0]/1e6 ),'\n'    
        'el_max = %0.4f [MW]' %( results['el_max'][0]/1e6 ),'\n'  
        'hs_max = %0.2f [T of H2]' %( results['hs_max'][0]/1e3 ),'\n'  
        'bat_capa = %0.2f [MWh]' %( results['bat_capa'][0]/3.6e9  ),'\n'  
        'bat_pmax = %0.2f [MW]' %( results['bat_pmax'][0]/1e6 )
        )


#transfer the data from 'results' to a dataframe for plotting: 
data_plot = pd.DataFrame(dict(t= np.arange(0,len(simparams['W'])),
#                               solar=simparams['S'],
                              wind=simparams['W'],
                              pv_out=results['pv_out']/1e6, #MW
                              w_out=results['w_out']/1e6, #MW
                              pc=results['pc']/1e6, #MW
                              he_out = results['he_out']*3.6, #T H2/hr
                              hr_out = results['hr_out']*3.6, #T H2/hr
                              sh = results['sh'][0:-1]/1e3, #T H2
                              R = results['r'][0:-1]/1e3, #T H2/hr 
                              L = results['L']*3.6, #T H2/hr
                              hs_out = (results['L']-results['hr_out'])*3.6, #T H2/hr
                              bat_e_stored = results['bat_e_stored'][0:-1]*3.6, #MWh
                              bat_pin = results['bat_pin']/1e6, #MW
                              bat_pout = results['bat_pout']/1e6 #MW
                             ))

#create a new column for datetime for plotting. This column is timestamped from the 
#beginning of 2022
data_plot['time'] =pd.to_datetime('2022-01-01') + pd.to_timedelta(data_plot.t, 'h')
data_plot.drop('t',axis=1,inplace=True)



CAPEX = 95.27 [M$] 
pv_max = 71.51 [MW] 
w_max = 0.00 [MW] 
el_max = 50.3633 [MW] 
hs_max = 134.52 [T of H2] 
bat_capa = 0.00 [MWh] 
bat_pmax = 47.61 [MW]


### Plot hydrogen flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.sh),
              name='H<sub>2</sub> Stored [T]',yaxis="y1"),
                )
# fig.add_trace(
#     go.Scatter(x=list(data_plot.time), y=list(data_plot.R),
#               name='R',yaxis="y1")
#                 )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.he_out),
              line_shape='hv',name='H<sub>2</sub> from Electrolyser [T/hr]',
               yaxis="y2", opacity=0.5)
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hs_out),
              line_shape='hv',name='H<sub>2</sub> from Storage [T/hr]',yaxis="y2")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hr_out),
              line_shape='hv',name='Unserved Load [T/hr]',yaxis="y2")
                )

# fig.update_layout(
#     yaxis1=dict(
#         range=[0, max(data_plot.sh)]
#     ),
#     yaxis2=dict(
#         range=[0, 0.4]
#     )   
# )
fig.update_yaxes(dict(title = '<b>T of H<sub>2</sub></b>',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = '<b>T of H<sub>2</sub>/hr</b>',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=10, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Hflow-CF=%s.png'%str(simparams['CF']))
fig.show()

### Plot power flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pv_out),
              line_shape='hv', name='PV Generation [MW]',
              line_color = 'orange', opacity= 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.w_out),
              line_shape='hv', name='Wind Generation [MW]',
              line_color = 'green', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pc),
              line_shape='hv', name='Curtailed Power [MW]',
              line_color = 'grey', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(0*data_plot.bat_pin),
              line_shape='hv', name='Battery Charging [MW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(0*data_plot.bat_pout),
              line_shape='hv', name='Battery Discharging [MW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_e_stored),
              line_shape='hv', name='Battery Energy [MWh]', yaxis="y2"),
                )


fig.update_yaxes(dict(title = 'MW',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = 'MWh',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=0, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Pflow-CF=%s.png'%str(simparams['CF']))

fig.show()

## Plot Solar Irradiance and wind speed

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.solar),
              line_shape='hv', name='Solar Irrad. [W/m<sup>2</sup>]',
               line_color = 'orange', opacity= 0.5, yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.wind),
              line_shape='hv', name='Wind Speed [m/s]',
               line_color = 'green', opacity= 0.5, yaxis="y2"),
                )
fig.update_yaxes(dict(title = 'W/m<sup>2</sup>'), secondary_y=False)
fig.update_yaxes(dict(title = 'm/s'), secondary_y=True)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=0, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\PVW-CF=%s.png'%str(simparams['CF']))

fig.show()

### Generate srw data file for wind model in PySam

In [1]:
from projdirs import datadir

import numpy as np
import pandas as pd
import json, io, requests
import PySAM.Pvwattsv8 as PVWatts, Windpower

import pandas as pd
import numpy as np
import os
from projdirs import datadir #load the path that contains the data files 
